In [11]:
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,classification_report,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
image_count = {}
default_image_size = tuple((64, 64))

data = []
c=-1
for root, _, files in os.walk('PlantVillage'):
    disease = os.path.basename(root)
    image_count[disease] = 0  
    c+=1
    if disease.lower().startswith(('tomato')):
        print(disease)
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.JPG') or file.endswith('.PNG') or file.endswith('JPEG') or file.endswith('jpeg') :
                image_path = os.path.join(root, file)

                if image_count[disease] >= 500:
                    continue

                # Read the image
                image = cv2.imread(image_path)
    #             image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, default_image_size)
                # Flatten the image to a 1D array and convert to NumPy array
                flattened_image = image.flatten().astype(int)
                # Store the flattened pixel values and disease name in data
                data.append([flattened_image, disease])

                # Increment the counter for the current disease
                image_count[disease] += 1

df = pd.DataFrame(data, columns=['image_pixels', 'disease'])

# Split the data into training and testing sets
X = np.vstack(df['image_pixels'].to_numpy())
y = df['disease']

# Encode disease labels using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the model

# Evaluate the model






Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold
Tomato_Septoria_leaf_spot
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato__Target_Spot
Tomato__Tomato_mosaic_virus
Tomato__Tomato_YellowLeaf__Curl_Virus


In [12]:
# model = RandomForestClassifier(n_estimators=100,random_state=42)
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)


In [13]:
import xgboost as xgb
# Now you can use XGBoost with the binary labels
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [14]:
# print(X_train)
# print(y_train)
# print(X_train.shape)


In [15]:
len(X_train[0])

12288

In [16]:
print(image_count)

{'PlantVillage': 0, 'Pepper__bell___Bacterial_spot': 0, 'Pepper__bell___healthy': 0, 'Potato___Early_blight': 0, 'Potato___healthy': 0, 'Potato___Late_blight': 0, 'Tomato_Bacterial_spot': 500, 'Tomato_Early_blight': 500, 'Tomato_healthy': 500, 'Tomato_Late_blight': 500, 'Tomato_Leaf_Mold': 500, 'Tomato_Septoria_leaf_spot': 500, 'Tomato_Spider_mites_Two_spotted_spider_mite': 500, 'Tomato__Target_Spot': 500, 'Tomato__Tomato_mosaic_virus': 373, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 500}


In [17]:
y_pred=model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7282051282051282


In [18]:
f1 = f1_score(y_test, y_pred, average='weighted')
f1

0.7265003971044967

In [19]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

array([[91,  5,  5,  2,  1,  1,  3,  1,  1,  2],
       [ 9, 57, 12,  1,  2,  2,  9,  6,  1,  3],
       [ 0, 21, 52,  6,  7,  0,  3,  3,  4,  5],
       [ 1,  1,  1, 76,  7,  2,  1,  0,  2,  1],
       [ 3,  1,  2,  8, 67,  2,  2,  4,  3,  1],
       [ 3,  6,  1,  1,  1, 70,  6,  2,  3,  0],
       [ 2,  4,  0,  1,  5, 12, 66,  0,  3,  2],
       [ 5,  4,  2,  0,  1,  8,  2, 87,  0,  1],
       [ 2,  0,  1,  6,  4,  1,  2,  1, 53,  0],
       [ 1,  4,  1,  2,  1,  1,  5,  0,  1, 91]], dtype=int64)